# **Text Analytics & Text Mining** by Syifa Afina
*Text Analytics* merupakan pendekatan dengan metodologi untuk mengekstrak pengetahuan dari data teks. Digunakan untuk kepentingan bisnis dan pengambilan keputusan lainnya.
> 
Sedangkan *Text Mining* merupakan alat yang digunakan pada *Text Analytics*.


## **Text Mining**
*Text Mining* merupakan proses melihat pola dan insight baru yang berasal dari banyaknya data yang pada awalnya tidak terstruktur (*unstructured data*). Pengolahan *Text Mining* membutuhkan proses yang dinamakan *Pre-Processing Data*.

## **Pre-Processing Data**
*Pre-Processing Data* merupakan proses untuk mempersiapkan data mentah (*raw data*) sebelum dilakukannya proses lain. Pada umunya, *pre-processing data* dilakukan dengan cara mengeliminasi data yang tidak sesuai atau mengubah data menjadi bentuk yang lebih mudah diproses oleh sistem.

Selain itu, praktikum kali ini akan membahas metode dalam *text mining* yaitu **Sentiment Analysis** dan **Topic Modelling**


1.   **Sentiment Analysis**

> Sentiment Analysis merupakan proses memahami dan mengelompokkan emosi (positif, negatif, dan netral) yang terdapat dalam tulisan menggunakan teknik Text Analytics. Data emosi tersebut dapat menjelaskan opini masyarakat mengenai produk, merek, layanan, politik, atau topik lainnya. Perusahaan, pemerintah, maupun bidang lainnya kemudian memanfaatkan data-data tersebut untuk membuat analisis pemasaran, review produk, umpan-balik produk, dan layanan masyarakat.

2.   **Topic Modelling**

> Topic Modelling merupakan pengelompokkan data teks berdasarkan suatu topik tertentu. Topic Modelling memiliki tujuan yang sama dengan classification tetapi menggunakan pendekatan berbeda. Topic Modelling tidak membutuhkan data berlabel (unsupervised learning) dan bekerja seperti clustering karena mengelompokkan data teks berdasarkan kemiripannya.








Praktikum kali ini menggunakan data tweet terkait Digital Currency sepanjang tahun 2020.

###**1. Install and Load Packages**

- packages tweet-preprocessor: memudahkan untuk membersihkan, mengurai, atau memberi token pada tweet.
- packages pyLDAvis: LDA (Latent Dirichlet Allocation) membantu dalam metode *Topic Modelling*

In [ ]:
# Install packages
! pip install tweet-preprocessor
! pip install pyLDAvis

In [ ]:
# Install pandas versi 1.3.0
  # (untuk mempermudah proses transformasi data)
! pip install pandas==1.3.0

In [ ]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt
import preprocessor as p
import numpy as np
import networkx as nx
import wordcloud
import nltk
import warnings
import itertools
import re
import os
import random
import pyLDAvis
import pyLDAvis.sklearn 

In [ ]:
# Import module
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from tqdm import tqdm
from nltk import bigrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Set parameter
 # (untuk mengabaikan 'filter warnings')
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

### **2. Import Data**
Dataset Digital Currency yang digunakan berasal dari github, sehingga import data langsung menggunakan link dataset

In [ ]:
# Import data
df = pd.read_csv("https://raw.githubusercontent.com/apriandito/digital-currency/main/data/tweet_digitalcurrency_text-sample.csv")

In [ ]:
# Lihat 5 baris pertama data
df.head()

In [ ]:
# Melihat ringkasan singkat dari dataset
df.info()

### **3. Text Preprocessing**
Proses pengolahan data

In [ ]:
# Pilih kolom 'TEXT' saja
tweet = df[['text']]

# Lihat 5 baris pertama data
tweet.head()

#### **a. Transformation**
Membersihkan dataset dari URLs, hashtags, mentions, RT & Fav, emoji, smileys, & number menggunakan packages tweet-preprocessor. Dilanjutkan dengan mengubah huruf pada dataset menjadi huruf kecil (lowercase)

In [ ]:
# Membuat fungsi transformasi tweet
 # dilanjutkan dengan mengubah huruf pada dataset menjadi lowercase

def transform_tweet(row):
  tweet = row['text']
  tweet = p.clean(tweet)
  tweet = str.lower(tweet)
  return tweet

In [ ]:
# Mengaplikasikan fungsi transformasi dengan membuat kolom baru di dataset
tweet['transformed'] = tweet.apply(transform_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning) 

#### **b. Tokenization**
Memotong teks menjadi unit yang lebih kecil yang disebut 'token'

In [ ]:
# Download Punkt
# Punkt merupakan alat 'Tokenizer' yang membagi teks menjadi unit-unit yang lebih kecil
nltk.download('punkt')

In [ ]:
# Membuat fungsi tokenization

def tokenize_tweet(row):
    tweet = row['transformed']
    tokens = nltk.word_tokenize(tweet)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [ ]:
# Mengaplikasikan fungsi tokenization dengan membuat kolom baru di dataset
tweet['tokenized'] = tweet.apply(tokenize_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning)
# kolom 'tokenized' merupakan teks setelah melalui proses tokenization

#### **c. Stemming**
Mengubah kata menjadi bentuk dasar umum (memotong/stem)

In [ ]:
# Membuat fungsi stemming
def stemming_tweet(row):
    list = row['tokenized']
    stemming_list = [PorterStemmer().stem(w) for w in list]
    return(stemming_list)

In [ ]:
# Mengaplikasikan fungsi stemming dengan membuat kolom baru di dataset
tweet['stemmed'] = tweet.apply(stemming_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning)
# kolom 'tokenized' merupakan teks setelah melalui proses tokenization
# kolom 'stemmed' merupakan teks setelah melalui proses stemming

#### **d. Lemmatization**
Mengubah kata menjadi bentuk dasar umum menggunakan analisis morfologi. Lemmatization biasanya lebih baik hasilnya dari Stemming. Stemming hanya memotong kata, sedangkan Lemmatization memotong kata dan menggunakan dictionary yang lebih lengkap.

In [ ]:
# Download Wordnet
nltk.download('wordnet')

In [ ]:
# Membuat fungsi lemmatization
def lemmatize_tweet(row):
    list = row['tokenized']
    lemmatize_list = [WordNetLemmatizer().lemmatize(w, pos= 'v') for w in list]
    return(lemmatize_list)

In [ ]:
# Mengaplikasikan fungsi lemmatization dengan membuat kolom baru di dataset
tweet['lemmatized'] = tweet.apply(lemmatize_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning)
# kolom 'tokenized' merupakan teks setelah melalui proses tokenization
# kolom 'stemmed' merupakan teks setelah melalui proses stemming
# kolom 'lemmatized' merupakan teks setelah melalui proses lemmatization

#### **e. Stopword Removal**
Menghapus kata yang tidak menambahkan arti pada kalimat.
Contoh:
> 
English: a, about, be, because, being, between, by, can't, do, during, from, have, he, she, him, her, it, is, i'm, you, too, they, this, etc.
> 
Indonesian: ada, agar, apa, nah, pula, pun, itu, dan, wah, yang, atau, dll.




In [ ]:
# Download stopword bahasa inggris
nltk.download('stopwords')
stops = set(stopwords.words("english"))     

In [ ]:
# Membuat fungsi stopword
def stopword_tweet(row):
    list = row['lemmatized']
    stopword_list = [w for w in list if not w in stops]
    return(stopword_list)

In [ ]:
# Mengaplikasikan fungsi Stopword dengan membuat kolom baru di dataset
tweet['stopword'] = tweet.apply(stopword_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning)
# kolom 'tokenized' merupakan teks setelah melalui proses tokenization
# kolom 'stemmed' merupakan teks setelah melalui proses stemming
# kolom 'lemmatized' merupakan teks setelah melalui proses lemmatization
# kolom 'stopword' merupakan teks setelah melalui proses stopword removal

#### **f. Rejoin Token**
Menggabungkan token / kata-kata yang telah sampai proses stopword menjadi kalimat utuh.

In [ ]:
# Membuat fungsi rejoin untuk mengembalikan sebagai kalimat utuh
def rejoin_tweet(row):
    list = row['stopword']
    joined_words = ( " ".join(list))
    return joined_words

In [ ]:
# Mengaplikasikan fungsi rejoin dengan membuat kolom baru di dataset
tweet['final'] = tweet.apply(rejoin_tweet, axis=1)

# Lihat 5 baris pertama data
tweet.head()

# kolom 'text' merupakan teks sebelum diubah
# kolom 'transformed' merupakan teks setelah melalui proses transformasi (cleaning)
# kolom 'tokenized' merupakan teks setelah melalui proses tokenization
# kolom 'stemmed' merupakan teks setelah melalui proses stemming
# kolom 'lemmatized' merupakan teks setelah melalui proses lemmatization
# kolom 'stopword' merupakan teks setelah melalui proses stopword removal
# kolom 'final' merupakan teks setelah melalui proses rejoin token

In [ ]:
# Final tweet yang sudah di proses
 # mengubah nama kolom "final" menjadi "text" untuk ditampilkan dengan variabel tweet_clean

tweet_clean = tweet[['final']]
tweet_clean = tweet_clean.rename(columns={'final': 'text'})

# Lihat 5 baris pertama data
tweet_clean.head()

### **4. WordCloud**
WordCloud merupakan bagian dari Content Description untuk melihat kemunculan dari kata-kata. Kata yang ukurannya paling besar yang muncul merupakan kata yang paling banyak kuantitasnya dalam dataset.

In [ ]:
# Visualisasi Word Cloud
text_wordcloud = " ".join(tweet for tweet in tweet_clean.text)

cloud = WordCloud(background_color='white').generate(text_wordcloud)

plt.figure(figsize=(10, 10), facecolor=None)
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

### **5. Sentiment Analysis**

In [ ]:
# Download corpus untuk sentiment analysis
nltk.download('vader_lexicon')

In [ ]:
# Sentiment Analysis
sid = SentimentIntensityAnalyzer()
listy = [] 
for index, row in tweet_clean.iterrows():
  ss = sid.polarity_scores(row['text'])
  listy.append(ss)
  
se = pd.Series(listy)
tweet_clean['polarity'] = se.values
display(tweet_clean.head(5))

In [ ]:
# Visualisasi Pie Chart
labels = ['negative', 'neutral', 'positive']
sizes  = [ss['neg'], ss['neu'], ss['pos']]
plt.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.axis('equal') 
plt.show()

### **6. Topic Modelling**

In [ ]:
# clone tambahan library dari github
! git clone https://github.com/machine-learning-ss/tm

# Set Data Directory
os.chdir('tm')

In [ ]:
# Melihat total loaded tweets

import MyLib as TS

Tweets = tweet_clean['text']
print('Total loaded tweets = {0}'.format(len(Tweets)))

In [ ]:
# Menentukan banyaknya jumlah topik dan kata-kata yang akan diambil
n_topics = 4
top_topics = 4
top_words = 10

In [ ]:
# Feature Extraction
count_vector = CountVectorizer(token_pattern = r'\b[a-zA-Z]{3,}\b') 
dtm_tf = count_vector.fit_transform(Tweets)
tf_terms = count_vector.get_feature_names()

In [ ]:
# Fungsi untuk mencari topic
lda_tf = LatentDirichletAllocation(n_components=n_topics, learning_method='online', random_state=0).fit(dtm_tf)

# Menampilkan Topik
vsm_topics = lda_tf.transform(dtm_tf); doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
print('In total there are {0} major topics, distributed as follows'.format(len(set(doc_topic))))
plt.hist(np.array(doc_topic), alpha=0.5); plt.show()
print('Printing top {0} Topics, with top {1} Words:'.format(top_topics, top_words))
TS.print_Topics(lda_tf, tf_terms, top_topics, top_words)

In [ ]:
# Visualisasi Topic Secara Interaktif
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, count_vector) 